## Read the `Object` tables

We start by showing how to read the `Object` tables using the Stripe 82 Object table as an example. Reading other tables (the `Source` and `ForcedSource`) will follow the exact same procedure. Totally, three object tables are provided:
- 'ObjectTable.parquet': The Object table for the entire data challenge including objects both extended Stripe 82 area and the XMM-LSS area.
- 's82ObjectTable.parquet': The table contains only the objects in the Stripe 82 region (no XMM-LSS coverage). 
- 'xmmlssObjectTable.parquet': The table contains the objects in the XMM-LSS area. 

All data sets provided in this data challenge are hosted in a root directory: 

>    /home/idies/workspace/Temporary/ywx649999311/LSST_AGN/Class_Training/DC/


__Note:__ The 'XMM-LSS' dataset is defined by both locations on the sky and the depth. Since our extended Stripe 82 region covers the XMM-LSS area, it is possible to find objects that doesn't exist in the XMM-LSS dataset (with much deeper photometry) but appears in the Stripe 82 dataset given we only did simply positional cross-match.

In [1]:
## commonly used modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os, sys
import yaml

pd.set_option('display.max_columns', 999)

In [2]:
## Data challenge root data dir
data_dir = "/home/idies/workspace/Temporary/ywx649999311/LSST_AGN/Class_Training/DC/"

### 0. Examine the data directory

In [4]:
!ls "/home/idies/workspace/Temporary/ywx649999311/LSST_AGN/Class_Training/DC/"

ForcedSourceTable.parquet	 s82ObjectTable.parquet
ObjectTable.parquet		 s82SourceTable.parquet
SourceTable.parquet		 xmmlssMultiWavelengthTable.parquet
lc_feats.yml			 xmmlssObjectTable.parquet
mw_cols.yml			 xmmlssSourceTable.parquet
s82MultiWavelengthTable.parquet


### 1. Read the Stripe 82 Object table
The object table follows the current version of the LSST Data Products document (LSE-163) as much as possible. We provide measurements of included objects in the following main catalogries:
- __Astromety__ -> ra, dec, proper motion and parallax
- __Photometry__ -> point and extended source photometry, in both AB magnitdues and fluxes (nJy)
- __Color__ -> Computed using the fluxes
- __Morphology__ -> 1 for extended and 0 for point-like
- __Light Curve Features__ -> Extrated on the SDSS light curves if matched
- __Redshift__ -> Both spectroscopic and photometric, wherever available

#### 1.1 Read the table

In [5]:
object_df = pd.read_parquet(os.path.join(data_dir, 's82ObjectTable.parquet'))
object_df.tail(3)

,ra,dec,psPm_ra,psPm_dec,psParallax,psFlux_u,psFlux_g,psFlux_r,psFlux_i,psFlux_z,psFlux_y,psFluxErr_u,psFluxErr_g,psFluxErr_r,psFluxErr_i,psFluxErr_z,psFluxErr_y,bdFlux_u,bdFlux_g,bdFlux_r,bdFlux_i,bdFlux_z,bdFlux_y,bdFluxErr_u,bdFluxErr_g,bdFluxErr_r,bdFluxErr_i,bdFluxErr_z,bdFluxErr_y,psMag_u,psMag_g,psMag_r,psMag_i,psMag_z,psMag_y,psMagErr_u,psMagErr_g,psMagErr_r,psMagErr_i,psMagErr_z,psMagErr_y,bdMag_u,bdMag_g,bdMag_r,bdMag_i,bdMag_z,bdMag_y,bdMagErr_u,bdMagErr_g,bdMagErr_r,bdMagErr_i,bdMagErr_z,bdMagErr_y,extendedness_u,extendedness_g,extendedness_r,extendedness_i,extendedness_z,extendedness_y,stdColor_0,stdColor_1,stdColor_2,stdColor_3,stdColor_4,stdColorErr_0,stdColorErr_1,stdColorErr_2,stdColorErr_3,stdColorErr_4,class,photoZ_pest,z,flags_u,flags_g,flags_r,flags_i,flags_z,flags_y,lcPeriodic[0]_g,lcPeriodic[0]_r,lcPeriodic[0]_i,lcPeriodic[1]_g,lcPeriodic[1]_r,lcPeriodic[1]_i,lcPeriodic[2]_g,lcPeriodic[2]_r,lcPeriodic[2]_i,lcPeriodic[3]_g,lcPeriodic[3]_r,lcPeriodic[3]_i,lcPeriodic[4]_u,lcPeriodic[4]_g,lcPeriodic[4]_r,lcPeriodic[4]_i,lcPeriodic[4]_z,lcPeriodic[5]_u,lcPeriodic[5]_g,lcPeriodic[5]_r,lcPeriodic[5]_i,lcPeriodic[5]_z,lcPeriodic[6]_u,lcPeriodic[6]_g,lcPeriodic[6]_r,lcPeriodic[6]_i,lcPeriodic[6]_z,lcPeriodic[7]_u,lcPeriodic[7]_g,lcPeriodic[7]_r,lcPeriodic[7]_i,lcPeriodic[7]_z,lcPeriodic[8]_u,lcPeriodic[8]_g,lcPeriodic[8]_r,lcPeriodic[8]_i,lcPeriodic[8]_z,lcPeriodic[9]_u,lcPeriodic[9]_g,lcPeriodic[9]_r,lcPeriodic[9]_i,lcPeriodic[9]_z,lcPeriodic[10]_u,lcPeriodic[10]_g,lcPeriodic[10]_r,lcPeriodic[10]_i,lcPeriodic[10]_z,lcPeriodic[11]_u,lcPeriodic[11]_g,lcPeriodic[11]_r,lcPeriodic[11]_i,lcPeriodic[11]_z,lcPeriodic[12]_u,lcPeriodic[12]_g,lcPeriodic[12]_r,lcPeriodic[12]_i,lcPeriodic[12]_z,lcPeriodic[13]_u,lcPeriodic[13]_g,lcPeriodic[13]_r,lcPeriodic[13]_i,lcPeriodic[13]_z,lcPeriodic[14]_u,lcPeriodic[14]_g,lcPeriodic[14]_r,lcPeriodic[14]_i,lcPeriodic[14]_z,lcPeriodic[15]_u,lcPeriodic[15]_g,lcPeriodic[15]_r,lcPeriodic[15]_i,lcPeriodic[15]_z,lcPeriodic[16]_u,lcPeriodic[16]_g,lcPeriodic[16]_r,lcPeriodic[16]_i,lcPeriodic[16]_z,lcPeriodic[17]_u,lcPeriodic[17]_g,lcPeriodic[17]_r,lcPeriodic[17]_i,lcPeriodic[17]_z,lcPeriodic[18]_u,lcPeriodic[18]_g,lcPeriodic[18]_r,lcPeriodic[18]_i,lcPeriodic[18]_z,lcPeriodic[19]_u,lcPeriodic[19]_g,lcPeriodic[19]_r,lcPeriodic[19]_i,lcPeriodic[19]_z,lcPeriodic[20]_u,lcPeriodic[20]_g,lcPeriodic[20]_r,lcPeriodic[20]_i,lcPeriodic[20]_z,lcPeriodic[21]_u,lcPeriodic[21]_g,lcPeriodic[21]_r,lcPeriodic[21]_i,lcPeriodic[21]_z,lcPeriodic[22]_u,lcPeriodic[22]_g,lcPeriodic[22]_r,lcPeriodic[22]_i,lcPeriodic[22]_z,lcPeriodic[23]_u,lcPeriodic[23]_g,lcPeriodic[23]_r,lcPeriodic[23]_i,lcPeriodic[23]_z,lcPeriodic[24]_u,lcPeriodic[24]_g,lcPeriodic[24]_r,lcPeriodic[24]_i,lcPeriodic[24]_z,lcPeriodic[25]_u,lcPeriodic[25]_g,lcPeriodic[25]_r,lcPeriodic[25]_i,lcPeriodic[25]_z,lcPeriodic[26]_u,lcPeriodic[26]_g,lcPeriodic[26]_r,lcPeriodic[26]_i,lcPeriodic[26]_z,lcPeriodic[27]_u,lcPeriodic[27]_g,lcPeriodic[27]_r,lcPeriodic[27]_i,lcPeriodic[27]_z,lcPeriodic[28]_u,lcPeriodic[28]_g,lcPeriodic[28]_r,lcPeriodic[28]_i,lcPeriodic[28]_z,lcPeriodic[29]_u,lcPeriodic[29]_g,lcPeriodic[29]_r,lcPeriodic[29]_i,lcPeriodic[29]_z,lcPeriodic[30]_u,lcPeriodic[30]_g,lcPeriodic[30]_r,lcPeriodic[30]_i,lcPeriodic[30]_z,lcPeriodic[31]_u,lcPeriodic[31]_g,lcPeriodic[31]_r,lcPeriodic[31]_i,lcPeriodic[31]_z,lcPeriodic[32]_u,lcPeriodic[32]_g,lcPeriodic[32]_r,lcPeriodic[32]_i,lcPeriodic[32]_z,lcNonPeriodic[0]_u,lcNonPeriodic[0]_g,lcNonPeriodic[0]_r,lcNonPeriodic[0]_i,lcNonPeriodic[0]_z,lcNonPeriodic[1]_u,lcNonPeriodic[1]_g,lcNonPeriodic[1]_r,lcNonPeriodic[1]_i,lcNonPeriodic[1]_z,lcNonPeriodic[2]_u,lcNonPeriodic[2]_g,lcNonPeriodic[2]_r,lcNonPeriodic[2]_i,lcNonPeriodic[2]_z,lcNonPeriodic[3]_u,lcNonPeriodic[3]_g,lcNonPeriodic[3]_r,lcNonPeriodic[3]_i,lcNonPeriodic[3]_z,lcNonPeriodic[4]_u,lcNonPeriodic[4]_g,lcNonPeriodic[4]_r,lcNonPeriodic[4]_i,lcNonPeriodic[4]_z,lcNonPeriodic[5]_u,lcNonPeriodic[5]_g,lcNonPeriodic[5]_r,lcNonPeriodic[5]_i,lcNonPeriodic[5]_z,lcNo

#### 1.2 Check column summaries

In [6]:
object_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 436990 entries, 0271386 to 1468021
Data columns (total 380 columns):
 #    Column               Non-Null Count   Dtype  
---   ------               --------------   -----  
 0    ra                   435901 non-null  float64
 1    dec                  435901 non-null  float64
 2    psPm_ra              432795 non-null  float64
 3    psPm_dec             432795 non-null  float64
 4    psParallax           133393 non-null  float64
 5    psFlux_u             376601 non-null  float64
 6    psFlux_g             423564 non-null  float64
 7    psFlux_r             425728 non-null  float64
 8    psFlux_i             425206 non-null  float64
 9    psFlux_z             426094 non-null  float64
 10   psFlux_y             381635 non-null  float64
 11   psFluxErr_u          376601 non-null  float64
 12   psFluxErr_g          423564 non-null  float64
 13   psFluxErr_r          425728 non-null  float64
 14   psFluxErr_i          425206 non-null  float64
 1

#### *__A Note on Column Names__*
Some entries in the LSST data release catalog as descibed by the __LSE_163__ are multi-dimensional; in the provided tables, we have flatten them. For example, the current LSST data model uses a 6x1 array to store `psFlux` for each of the six bands (ugrizy), in the provided `Object` table, we used `psFlux_u` to represent the `psFlux` measurement in the 'u' band. For the `lcPeriodic` and `lcNonPeriodic` columns, we used square brakets ('[ ]') to emphasize the index of the correpsoinding feature. 

## 2. Light curvce feature definitions
You can get more infomration about how the light curve features are extrated from the notebooks included in the 'docs' folder. To just get a mapping from the index to the corresponding feature, you can read the included 'lc_feats.yml' file as below.

In [7]:
lc_feats_dfs = yaml.safe_load(open(os.path.join(data_dir, 'lc_feats.yml')))

#### 2.1 `lcPeriodic`

In [8]:
lc_feats_dfs['lcPeriodic']

{0: 'alpha_1 from the CARMA(2,1)/DHO fit to the light curve (in flux)',
 1: 'alpha_2 from the CARMA(2,1)/DHO fit to the light curve (in flux)',
 2: 'beta_0 from the CARMA(2,1)/DHO fit to the light curve (in flux)',
 3: 'beta_1 from the CARMA(2,1)/DHO fit to the light curve (in flux)',
 4: "freq1_harmonics_amplitude_0, see 'arXiv:1101.1959'",
 5: "freq1_harmonics_amplitude_1, see 'arXiv:1101.1959'",
 6: "freq1_harmonics_amplitude_2, see 'arXiv:1101.1959'",
 7: "freq1_harmonics_amplitude_3, see 'arXiv:1101.1959'",
 8: "freq1_harmonics_freq_0, see 'arXiv:1101.1959'",
 9: "freq1_harmonics_rel_phase_1, see 'arXiv:1101.1959'",
 10: "freq1_harmonics_rel_phase_2, see 'arXiv:1101.1959'",
 11: "freq1_harmonics_rel_phase_3, see 'arXiv:1101.1959'",
 12: "freq2_harmonics_amplitude_0, see 'arXiv:1101.1959'",
 13: "freq2_harmonics_amplitude_1, see 'arXiv:1101.1959'",
 14: "freq2_harmonics_amplitude_2, see 'arXiv:1101.1959'",
 15: "freq2_harmonics_amplitude_3, see 'arXiv:1101.1959'",
 16: "freq2_harmo

#### 2.2 `lcNonPeriodic`

In [9]:
lc_feats_dfs['lcNonPeriodic']

{0: 'Amplitude -> half the difference between the maximum and the minimum magnitude',
 1: 'beyond1std -> percentage of points beyond one st. dev. from the weighted mean',
 2: 'flux_percentile_ratio_mid20 -> ratio of magnitude percentiles (60th–40th) over (95th–5th)',
 3: 'flux_percentile_ratio_mid35 -> ratio of magnitude percentiles (67.5th–32.5th) over (95th–5th)',
 4: 'flux_percentile_ratio_mid50 -> ratio of magnitude percentiles (75th–25th) over (95th–5th)',
 5: 'flux_percentile_ratio_mid65 -> ratio of magnitude percentiles (82.5th–17.5th) over (95th–5th)',
 6: 'flux_percentile_ratio_mid80 -> ratio of magnitude percentiles (90th–10th) over (95th–5th)',
 7: 'linear_trend -> slope of a linear fit to the light-curve magnitude',
 8: 'max_slope -> maximum absolute magnitude slope between two consecutive observations',
 9: 'median_absolute_deviation -> median discrepancy of the magnitudes from the median magnitude',
 10: 'median_buffer_range_percentage -> percentage of magnitudes within 1